In [1]:
# Import relevant functionality
from langchain_anthropic import ChatAnthropic
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

# Create the agent
memory = MemorySaver()
model = ChatAnthropic(model_name="claude-3-sonnet-20240229")
search = TavilySearchResults(max_results=2)
tools = [search]
agent_executor = create_react_agent(model, tools, checkpointer=memory)

ModuleNotFoundError: No module named 'langchain_anthropic'

In [5]:
import getpass
import os

os.environ["TAVILY_API_KEY"] = getpass.getpass()

In [7]:
from langchain_community.tools.tavily_search import TavilySearchResults

# a search engine. 

search = TavilySearchResults(max_results=2)
search_results = search.invoke("what is the weather in Mexico City")
print(search_results)
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

[{'title': 'Weather in Mexico City', 'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'Mexico City', 'region': 'The Federal District', 'country': 'Mexico', 'lat': 19.4285, 'lon': -99.1277, 'tz_id': 'America/Mexico_City', 'localtime_epoch': 1741788931, 'localtime': '2025-03-12 08:15'}, 'current': {'last_updated_epoch': 1741788000, 'last_updated': '2025-03-12 08:00', 'temp_c': 14.6, 'temp_f': 58.4, 'is_day': 1, 'condition': {'text': 'Sunny', 'icon': '//cdn.weatherapi.com/weather/64x64/day/113.png', 'code': 1000}, 'wind_mph': 2.2, 'wind_kph': 3.6, 'wind_degree': 201, 'wind_dir': 'SSW', 'pressure_mb': 1019.0, 'pressure_in': 30.08, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 63, 'cloud': 0, 'feelslike_c': 14.7, 'feelslike_f': 58.4, 'windchill_c': 14.7, 'windchill_f': 58.4, 'heatindex_c': 14.6, 'heatindex_f': 58.4, 'dewpoint_c': 6.4, 'dewpoint_f': 43.5, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 1.1, 'gust_mph': 2.7, 'gust_kph': 4.3}}", 'score': 0.909123}, {'title': '

In [9]:
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

from langchain.chat_models import init_chat_model

model = init_chat_model("gpt-4", model_provider="openai")

In [10]:
# calling a model

from langchain_core.messages import HumanMessage

response = model.invoke([HumanMessage(content="hi!")])
response.content

'Hello! How can I assist you today?'

In [11]:
model_with_tools = model.bind_tools(tools)

In [12]:
response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: Hello! How can I assist you today?
ToolCalls: []


In [16]:
# expected call to tool
response = model_with_tools.invoke([HumanMessage(content="What's the weather in Barcelona?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Barcelona'}, 'id': 'call_rooKv9DOOz6wN7q5KzUgmvE7', 'type': 'tool_call'}]


In [14]:
tools

[TavilySearchResults(max_results=2, api_wrapper=TavilySearchAPIWrapper(tavily_api_key=SecretStr('**********')))]